# Deep Learning Grid Search


In [2]:
import sys
import pandas
import numpy
import sklearn
import keras

In [6]:
import pandas as pd
import numpy as np
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv('pima-indians-diabetes.csv', names = names)

In [7]:
# Describe the dataset
df.describe()

,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
df[df['glucose_concentration'] == 0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
84,1,0.0,48.0,20.0,0.0,24.7,0.140,22.0,0.0
191,1,0.0,74.0,20.0,23.0,27.7,0.299,21.0,0.0
351,1,0.0,68.0,35.0,0.0,32.0,0.389,22.0,0.0
358,5,0.0,80.0,32.0,0.0,41.0,0.346,37.0,1.0
511,6,0.0,68.0,41.0,0.0,39.0,0.727,41.0,1.0


In [9]:
# Preprocess the data, mark zero values as NaN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()

,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
# Drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in df
df.describe()

,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [11]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset.shape)

(392, 9)


In [12]:
# split into input (X) and an output (Y)
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [13]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 8)
(392,)
[0 1 1 1 1]


In [14]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

/Users/junaid/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [16]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

/Users/junaid/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [17]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [18]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 1)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Epoch 1/10
261/261 [==============================] - 2s 7ms/step - loss: 0.6297 - acc: 0.6935
Epoch 2/10
261/261 [==============================] - 0s 245us/step - loss: 0.4854 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 277us/step - loss: 0.4422 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 289us/step - loss: 0.4268 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 275us/step - loss: 0.4151 - acc: 0.8046
Epoch 6/10
261/261 [==============================] - 0s 302us/step - loss: 0.4021 - acc: 0.8238
Epoch 7/10
261/261 [==============================] - 0s 294us/step - loss: 0.3933 - acc: 0.8352
Epoch 8/10
261/261 [==============================] - 0s 286us/step - loss: 0.3917 - acc: 0.8199
Epoch 9/10
261/261 [==============================] - 0s 299us/step - loss: 0.3929 - acc: 0.8391
Epoch 10/10
261/261 [==============================] - 0s 194us/step


261/261 [==============================] - 0s 149us/step
Epoch 1/50
261/261 [==============================] - 2s 9ms/step - loss: 0.6605 - acc: 0.6935
Epoch 2/50
261/261 [==============================] - 0s 258us/step - loss: 0.5319 - acc: 0.7931
Epoch 3/50
261/261 [==============================] - 0s 225us/step - loss: 0.4509 - acc: 0.7931
Epoch 4/50
261/261 [==============================] - 0s 222us/step - loss: 0.4448 - acc: 0.7854
Epoch 5/50
261/261 [==============================] - 0s 218us/step - loss: 0.4280 - acc: 0.8123
Epoch 6/50
261/261 [==============================] - 0s 291us/step - loss: 0.4326 - acc: 0.8084
Epoch 7/50
261/261 [==============================] - 0s 331us/step - loss: 0.4159 - acc: 0.8008
Epoch 8/50
261/261 [==============================] - 0s 306us/step - loss: 0.4099 - acc: 0.8046
Epoch 9/50
261/261 [==============================] - 0s 241us/step - loss: 0.4053 - acc: 0.8123
Epoch 10/50
261/261 [==============================] - 0s 279us/step - l

262/262 [==============================] - 0s 240us/step - loss: 0.3772 - acc: 0.8397
Epoch 33/50
262/262 [==============================] - 0s 249us/step - loss: 0.3716 - acc: 0.8244
Epoch 34/50
262/262 [==============================] - 0s 258us/step - loss: 0.3766 - acc: 0.8321
Epoch 35/50
262/262 [==============================] - 0s 406us/step - loss: 0.3801 - acc: 0.8244
Epoch 36/50
262/262 [==============================] - 0s 471us/step - loss: 0.3662 - acc: 0.8359
Epoch 37/50
262/262 [==============================] - 0s 389us/step - loss: 0.3643 - acc: 0.8359
Epoch 38/50
262/262 [==============================] - 0s 296us/step - loss: 0.3624 - acc: 0.8473
Epoch 39/50
262/262 [==============================] - 0s 273us/step - loss: 0.3646 - acc: 0.8397
Epoch 40/50
262/262 [==============================] - 0s 268us/step - loss: 0.3590 - acc: 0.8397
Epoch 41/50
262/262 [==============================] - 0s 308us/step - loss: 0.3536 - acc: 0.8435
Epoch 42/50
262/262 [===========

261/261 [==============================] - 0s 213us/step - loss: 0.2627 - acc: 0.8812
Epoch 64/100
261/261 [==============================] - 0s 224us/step - loss: 0.3021 - acc: 0.8582
Epoch 65/100
261/261 [==============================] - 0s 206us/step - loss: 0.2751 - acc: 0.8736
Epoch 66/100
261/261 [==============================] - 0s 200us/step - loss: 0.2620 - acc: 0.8927
Epoch 67/100
261/261 [==============================] - 0s 206us/step - loss: 0.2540 - acc: 0.8927
Epoch 68/100
261/261 [==============================] - 0s 202us/step - loss: 0.2493 - acc: 0.8927
Epoch 69/100
261/261 [==============================] - 0s 216us/step - loss: 0.2487 - acc: 0.9004
Epoch 70/100
261/261 [==============================] - 0s 208us/step - loss: 0.2533 - acc: 0.8966
Epoch 71/100
261/261 [==============================] - 0s 201us/step - loss: 0.2525 - acc: 0.8927
Epoch 72/100
261/261 [==============================] - 0s 202us/step - loss: 0.2414 - acc: 0.9042
Epoch 73/100
261/261 [=

261/261 [==============================] - 0s 232us/step - loss: 0.3137 - acc: 0.8812
Epoch 45/100
261/261 [==============================] - 0s 226us/step - loss: 0.3289 - acc: 0.8621
Epoch 46/100
261/261 [==============================] - 0s 279us/step - loss: 0.3735 - acc: 0.8506
Epoch 47/100
261/261 [==============================] - 0s 225us/step - loss: 0.3737 - acc: 0.8391
Epoch 48/100
261/261 [==============================] - 0s 226us/step - loss: 0.3583 - acc: 0.8544
Epoch 49/100
261/261 [==============================] - 0s 245us/step - loss: 0.3440 - acc: 0.8582
Epoch 50/100
261/261 [==============================] - 0s 228us/step - loss: 0.3281 - acc: 0.8697
Epoch 51/100
261/261 [==============================] - 0s 219us/step - loss: 0.3217 - acc: 0.8697
Epoch 52/100
261/261 [==============================] - 0s 242us/step - loss: 0.3175 - acc: 0.8736
Epoch 53/100
261/261 [==============================] - 0s 326us/step - loss: 0.3117 - acc: 0.8736
Epoch 54/100
261/261 [=

...

261/261 [==============================] - 0s 85us/step - loss: 0.3056 - acc: 0.8812
Epoch 59/100
261/261 [==============================] - 0s 96us/step - loss: 0.3113 - acc: 0.8736
Epoch 60/100
261/261 [==============================] - 0s 85us/step - loss: 0.3061 - acc: 0.8774
Epoch 61/100
261/261 [==============================] - 0s 92us/step - loss: 0.3058 - acc: 0.8774
Epoch 62/100
261/261 [==============================] - 0s 79us/step - loss: 0.3001 - acc: 0.8774
Epoch 63/100
261/261 [==============================] - 0s 89us/step - loss: 0.2996 - acc: 0.8736
Epoch 64/100
261/261 [==============================] - 0s 86us/step - loss: 0.2999 - acc: 0.8736
Epoch 65/100
261/261 [==============================] - 0s 87us/step - loss: 0.2916 - acc: 0.8812
Epoch 66/100
261/261 [==============================] - 0s 92us/step - loss: 0.2920 - acc: 0.8774
Epoch 67/100
261/261 [==============================] - 0s 82us/step - loss: 0.2942 - acc: 0.8774
Epoch 68/100
261/261 [===========

262/262 [==============================] - 0s 94us/step - loss: 0.3963 - acc: 0.8092
Epoch 41/100
262/262 [==============================] - 0s 84us/step - loss: 0.3950 - acc: 0.8092
Epoch 42/100
262/262 [==============================] - 0s 84us/step - loss: 0.3939 - acc: 0.8053
Epoch 43/100
262/262 [==============================] - 0s 92us/step - loss: 0.3955 - acc: 0.8092
Epoch 44/100
262/262 [==============================] - 0s 76us/step - loss: 0.3899 - acc: 0.8206
Epoch 45/100
262/262 [==============================] - 0s 82us/step - loss: 0.3884 - acc: 0.8282
Epoch 46/100
262/262 [==============================] - 0s 89us/step - loss: 0.3878 - acc: 0.8168
Epoch 47/100
262/262 [==============================] - 0s 76us/step - loss: 0.3907 - acc: 0.8244
Epoch 48/100
262/262 [==============================] - 0s 95us/step - loss: 0.3865 - acc: 0.8206
Epoch 49/100
262/262 [==============================] - 0s 88us/step - loss: 0.3824 - acc: 0.8206
Epoch 50/100
262/262 [===========

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.8min finished


Epoch 1/100
392/392 [==============================] - 3s 7ms/step - loss: 0.6575 - acc: 0.6939
Epoch 2/100
392/392 [==============================] - 0s 149us/step - loss: 0.5065 - acc: 0.7755
Epoch 3/100
392/392 [==============================] - 0s 134us/step - loss: 0.4539 - acc: 0.7730
Epoch 4/100
392/392 [==============================] - 0s 126us/step - loss: 0.4439 - acc: 0.7806
Epoch 5/100
392/392 [==============================] - 0s 128us/step - loss: 0.4365 - acc: 0.7730
Epoch 6/100
392/392 [==============================] - 0s 127us/step - loss: 0.4331 - acc: 0.7781
Epoch 7/100
392/392 [==============================] - 0s 127us/step - loss: 0.4360 - acc: 0.7857
Epoch 8/100
392/392 [==============================] - 0s 129us/step - loss: 0.4292 - acc: 0.7908
Epoch 9/100
392/392 [==============================] - 0s 127us/step - loss: 0.4281 - acc: 0.7857
Epoch 10/100
392/392 [==============================] - 0s 129us/step - loss: 0.4251 - acc: 0.8087
Epoch 11/100
392/392 

392/392 [==============================] - 0s 127us/step - loss: 0.3255 - acc: 0.8776
Epoch 84/100
392/392 [==============================] - 0s 134us/step - loss: 0.3259 - acc: 0.8673
Epoch 85/100
392/392 [==============================] - 0s 133us/step - loss: 0.3250 - acc: 0.8699
Epoch 86/100
392/392 [==============================] - 0s 127us/step - loss: 0.3180 - acc: 0.8724
Epoch 87/100
392/392 [==============================] - 0s 127us/step - loss: 0.3209 - acc: 0.8648
Epoch 88/100
392/392 [==============================] - 0s 130us/step - loss: 0.3321 - acc: 0.8673
Epoch 89/100
392/392 [==============================] - 0s 130us/step - loss: 0.3234 - acc: 0.8724
Epoch 90/100
392/392 [==============================] - 0s 127us/step - loss: 0.3242 - acc: 0.8622
Epoch 91/100
392/392 [==============================] - 0s 128us/step - loss: 0.3191 - acc: 0.8724
Epoch 92/100
392/392 [==============================] - 0s 127us/step - loss: 0.3310 - acc: 0.8622
Epoch 93/100
392/392 [=

In [23]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.74809161397337, total=  11.4s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.5s remaining:    0.0s


[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.778625960113438, total=  10.9s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   22.4s remaining:    0.0s


[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.8461538553237915, total=  10.8s
[CV] dropout_rate=0.0, learn_rate=0.01 ...............................


...

[CV]  dropout_rate=0.0, learn_rate=0.1, score=0.8153846126336318, total=  11.0s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.6min remaining:    0.0s


[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.725190845154624, total=  11.7s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.7633587900008864, total=  11.9s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.8384615366275494, total=  12.1s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.7480916039634297, total=  12.0s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.7557251903846973, total=  11.8s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.8461538553237915, total=  12.2s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV]  dropout_rate=0.1, learn_rate=0.1, score=0.7251908392396592, total=  12.2s
[CV] d

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  5.4min finished


Best: 0.7908163351976142, using {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7908163351976142 (0.04092945245626919) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7882653032334483 (0.0432823723183419) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7500000015205267 (0.049716980052564065) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7755102090993706 (0.04700773782551412) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7831632684992285 (0.044480326005227895) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7653061276187703 (0.03078572101203622) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.783163269107439 (0.045347471123823514) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7653061201681897 (0.04098916677601005) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7346938770948624 (0.03277710189665947) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [24]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV]  activation=softmax, init=uniform, score=0.7557252035796187, total=  13.3s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.3s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.7557252003946378, total=  12.5s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.8s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.8153846218035772, total=  13.3s
[CV] activation=softmax, init=normal .................................


...

[CV]  activation=softmax, init=zero, score=0.699999988079071, total=  13.0s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.0min remaining:    0.0s


[CV]  activation=relu, init=uniform, score=0.7328244347608727, total=  13.0s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.748091610788389, total=  12.9s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.8230769221599286, total=  14.3s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.7175572455384349, total=  14.4s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.7709923705071894, total=  14.4s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.8384615503824674, total=  13.9s
[CV] activation=relu, init=zero ......................................
[CV]  activation=relu, init=zero, score=0.6106870242657553, total=  13.9s
[CV] activation=relu, init=zero ..........

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  8.8min finished


Best: 0.7933673531729348, using {'activation': 'tanh', 'init': 'normal'}
0.7755102136609505 (0.028087641954935023) with: {'activation': 'softmax', 'init': 'uniform'}
0.7295918416003792 (0.08860773018104312) with: {'activation': 'softmax', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'softmax', 'init': 'zero'}
0.7678571475707755 (0.03939442059435707) with: {'activation': 'relu', 'init': 'uniform'}
0.7755102072747386 (0.04943085836731995) with: {'activation': 'relu', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'relu', 'init': 'zero'}
0.7831632721484924 (0.02879958859349298) with: {'activation': 'tanh', 'init': 'uniform'}
0.7933673531729348 (0.032371719468446684) with: {'activation': 'tanh', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'tanh', 'init': 'zero'}
0.7933673531729348 (0.037313655933021314) with: {'activation': 'linear', 'init': 'uniform'}
0.7908163291155076 (0.0337061659

In [25]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................
[CV] ... neuron1=4, neuron2=2, score=0.7709923645922245, total=  15.8s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.8s remaining:    0.0s


[CV] ... neuron1=4, neuron2=2, score=0.7633587900008864, total=  16.9s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   32.8s remaining:    0.0s


[CV] ... neuron1=4, neuron2=2, score=0.8230769267449012, total=  16.0s
[CV] neuron1=4, neuron2=4 ............................................


...

[CV] .... neuron1=4, neuron2=8, score=0.830769236271198, total=  17.3s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.5min remaining:    0.0s


[CV] ... neuron1=8, neuron2=2, score=0.7633587840859216, total=  18.8s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ... neuron1=8, neuron2=2, score=0.7633587900008864, total=  18.3s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ... neuron1=8, neuron2=2, score=0.8384615457974948, total=  19.1s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.7633587749859759, total=  18.9s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.7633587900008864, total=  17.4s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.8384615457974948, total=  17.5s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ... neuron1=8, neuron2=8, score=0.7633587840859216, total=  18.2s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  8.1min finished


Best: 0.7908163351976142, using {'neuron1': 16, 'neuron2': 2}
0.785714290123813 (0.02650267677503863) with: {'neuron1': 4, 'neuron2': 2}
0.7882653126607135 (0.019356087898682556) with: {'neuron1': 4, 'neuron2': 4}
0.785714290123813 (0.03173682706863349) with: {'neuron1': 4, 'neuron2': 8}
0.7882653141812402 (0.03535836258888925) with: {'neuron1': 8, 'neuron2': 2}
0.7882653111401869 (0.03535836473101593) with: {'neuron1': 8, 'neuron2': 4}
0.7857142931648663 (0.031736824926506764) with: {'neuron1': 8, 'neuron2': 8}
0.7908163351976142 (0.03897990025127174) with: {'neuron1': 16, 'neuron2': 2}
0.7908163306360342 (0.03370616199600475) with: {'neuron1': 16, 'neuron2': 4}
0.7857142931648663 (0.031736824926506764) with: {'neuron1': 16, 'neuron2': 8}


In [26]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [27]:
print(y_pred.shape)

(392, 1)


In [28]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [29]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7806122448979592
             precision    recall  f1-score   support

          0       0.81      0.89      0.84       262
          1       0.71      0.57      0.63       130

avg / total       0.77      0.78      0.77       392

